In [1]:
from nltk.corpus import stopwords
import string, re
from os import listdir
from pickle import load, dump
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all the text
    text = file.read()
    # close the file
    file.close()
    return text

# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

# quick trial
filename = 'C:\\Users\\Aaron\\Downloads\\txt_sentoken\\pos\\cv000_29590.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

Using TensorFlow backend.


['films', 'adapted', 'comic', 'books', 'plenty', 'success', 'whether', 'theyre', 'superheroes', 'batman', 'superman', 'spawn', 'geared', 'toward', 'kids', 'casper', 'arthouse', 'crowd', 'ghost', 'world', 'theres', 'never', 'really', 'comic', 'book', 'like', 'hell', 'starters', 'created', 'alan', 'moore', 'eddie', 'campbell', 'brought', 'medium', 'whole', 'new', 'level', 'mid', 'series', 'called', 'watchmen', 'say', 'moore', 'campbell', 'thoroughly', 'researched', 'subject', 'jack', 'ripper', 'would', 'like', 'saying', 'michael', 'jackson', 'starting', 'look', 'little', 'odd', 'book', 'graphic', 'novel', 'pages', 'long', 'includes', 'nearly', 'consist', 'nothing', 'footnotes', 'words', 'dont', 'dismiss', 'film', 'source', 'get', 'past', 'whole', 'comic', 'book', 'thing', 'might', 'find', 'another', 'stumbling', 'block', 'hells', 'directors', 'albert', 'allen', 'hughes', 'getting', 'hughes', 'brothers', 'direct', 'seems', 'almost', 'ludicrous', 'casting', 'carrot', 'top', 'well', 'anythi

In [2]:
# load all docs in a directory
def process_docs(directory, is_train):
    documents = list()
    # walk through all files in the folder
    for filename in listdir(directory):
        # skip any reviews in the test set
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
        # create the full path of the file to open
        path = directory + '\\' + filename
        # load the doc
        doc = load_doc(path)
        # clean doc
        tokens = clean_doc(doc)
        # add to list
        documents.append(tokens)
    return documents

# load and clean a dataset
def load_clean_dataset(is_train):
    # load documents
    neg = process_docs('C:\\Users\\Aaron\\Downloads\\txt_sentoken\\neg',is_train)
    pos = process_docs('C:\\Users\\Aaron\\Downloads\\txt_sentoken\\pos',is_train)
    docs = neg + pos
    # prepare labels
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
    return docs, labels

In [3]:
# save a dataset to file
def save_dataset(dataset, filename):
    dump(dataset, open(filename, 'wb'))
    print('Saved %s' % filename)

In [4]:
# load and clean all reviews
X_train, y_train = load_clean_dataset(True)
X_test, y_test = load_clean_dataset(False)
# save training datasets
save_dataset([X_train, y_train], 'train.pkl')
save_dataset([X_test, y_test], 'test.pkl')

Saved train.pkl
Saved test.pkl


# Develop Multichannel Model

In [5]:
# load a clean dataset
def load_dataset(filename):
    return load(open(filename, 'rb'))

In [6]:
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [16]:
# calculate maximium document length
def max_length(lines):
    return max([len(s) for s in lines])

In [8]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [20]:
# building the model

# define the model
def define_model(length, vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = Dropout(.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpertation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy',optimizer='adam', 
                  metrics=['accuracy'])
    # summarize
    model.summary()
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [21]:
# load training dataset
trainLines, trainLabels = load_dataset('train.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max length document
length = max_length(trainLines)
print('Max document length: %d' % length)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# encode data
X_train = encode_text(tokenizer, trainLines, length)
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([X_train,X_train,X_train], trainLabels, epochs=7, batch_size=16)
# save the model
model.save('CNN_NLP_Model.h5')

Max document length: 1380
Vocabulary size: 44277
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 1380)         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 1380)         0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 1380)         0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 1380, 100)    4427700     input_7[0][0]                    
___________________________________________

C:\Users\Aaron\anaconda3\envs\DeepLearning\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
1800/1800 [==============================] - 25s 14ms/step - loss: 0.6920 - accuracy: 0.5400
Epoch 2/7
1800/1800 [==============================] - 24s 14ms/step - loss: 0.4571 - accuracy: 0.8122
Epoch 3/7
1800/1800 [==============================] - 24s 13ms/step - loss: 0.0428 - accuracy: 0.9911
Epoch 4/7
1800/1800 [==============================] - 24s 13ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 5/7
1800/1800 [==============================] - 24s 13ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 6/7
1800/1800 [==============================] - 24s 13ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 7/7
1800/1800 [==============================] - 24s 13ms/step - loss: 7.4239e-04 - accuracy: 1.0000


In [24]:
# Evaluating the model
# load test dataset
testLines, testLabels = load_dataset('test.pkl')

# encode data
X_test = encode_text(tokenizer, testLines, length)
print(X_train.shape, X_test.shape)

# evaluate model on training dataset
_, acc = model.evaluate([X_train,X_train,X_train], trainLabels, verbose=0)
print('Train Accuracy: %.2f' % (acc*100))
# evaluate model on test dataset dataset
_, acc = model.evaluate([X_test,X_test,X_test], testLabels, verbose=0)
print('Test Accuracy: %.2f' % (acc*100))

(1800, 1380) (200, 1380)
Train Accuracy: 100.00
Test Accuracy: 85.00
